In [ ]:
import pandas as pd
import tqdm

In [ ]:
hic = pd.read_csv('/home/micl/workspace/lmh_data/Lee2019/BandNorm/bandnorm_result.txt', sep='\t')
hic

In [ ]:
hic_group = hic.groupby('cell')
_hic = pd.DataFrame()
for key in tqdm.notebook.tqdm(hic_group.groups, desc='group'):
    _data = hic_group.get_group(key)
    _data = _data.pivot_table(index=['chrom', 'binA'], values='BandNorm').T
    _data.index = [key]
    _hic = pd.concat([_hic, _data], join='outer')
hic = _hic
hic = hic.fillna(0)
hic

In [ ]:
_hic = anndata.AnnData(X=hic)
_hic.obs['cell_type'] = _hic.obs_names.str.split(r"[:_]").map(lambda x: x[5])

sc.pp.highly_variable_genes(_hic, n_top_genes=2000, flavor="seurat_v3")
sc.pp.normalize_total(_hic)
sc.pp.log1p(_hic)
sc.pp.scale(_hic)
sc.tl.pca(_hic, n_comps=100, svd_solver="auto")

sc.pp.neighbors(_hic, metric="cosine")
sc.tl.umap(_hic)
sc.pl.umap(_hic, color="cell_type")